# Importsand preparations

In [1]:
import pandas as pd
import torch
import os
import numpy as np
import datasets
import transformers
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

from datasets import load_dataset, DatasetDict

/itf-fi-ml/home/krimhau/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# !watch -n 0.5 nvidia-smi

In [3]:
print(f'PyTorch version: {torch.__version__}')  # 1.9.1+cu111
print(f'CUDA version: {torch.version.cuda}')  # 11.1
print(f'cuDNN version: {torch.backends.cudnn.version()}')  # 8005
print(f'Current device: {torch.cuda.current_device()}')  # 0
print(f'Is cuda available: {torch.cuda.is_available()}')  # TRUE

PyTorch version: 2.1.0+cu121
CUDA version: 12.1
cuDNN version: 8902
Current device: 0
Is cuda available: True


In [4]:
print(f'Transformers version: {transformers.__version__}')
print(f'Datasets version: {datasets.__version__}')

Transformers version: 4.35.0
Datasets version: 2.14.6


In [5]:
# Prevent a warning related to the tokenization process in the transformers library. 
os.environ["TOKENIZERS_PARALLELISM"] = "False"
# Makes CUDA operations synchronous
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [6]:
# Find the GPU with the least memory usage.
!nvidia-smi

Wed Jan 31 13:54:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  | 00000000:01:00.0 Off |                  N/A |
| 22%   32C    P2              58W / 250W |  10915MiB / 11264MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    # free unreferenced tensors from the GPU memory.
    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()   

Initial GPU Usage
| ID | GPU  | MEM |
-------------------
|  0 |   1% | 97% |
|  1 |   0% |  0% |
|  2 |   0% |  0% |
|  3 |   0% | 93% |
|  4 | 100% |  0% |
|  5 | 100% |  0% |
|  6 | 100% |  0% |
|  7 |   0% | 11% |
GPU Usage after emptying the cache
| ID | GPU  | MEM |
-------------------
|  0 |   1% | 97% |
|  1 |   0% |  0% |
|  2 |   0% |  0% |
|  3 |   0% | 93% |
|  4 | 100% |  0% |
|  5 | 100% |  0% |
|  6 | 100% |  0% |
|  7 |   0% | 12% |


In [8]:
# Smaller and faster than bert.
model_ckpt = "distilbert-base-uncased"

epochs = 5 #Number of full cycles through the training set.
num_labels = 2 #Number of labels, high, med, low priority.
learning_rate = 5e-5 # Rate the model updates based on the data its trained on.
train_batch_size = 16 # Number of training examples in one iteration.
eval_batch_size = 32 # Number evaluation examples in on iteration.
save_strategy = "no" # Should the model be saved automatically during training.
save_steps = 500 # How often to save the model during training. No effect since no over.
logging_steps = 100
model_dir = "./model" #Where to save model

# Use early stopping to prevent overfitting
#load_best_model_at_end=True
#metric_for_best_model="eval_loss"
#greater_is_better=False

Load dataset from huggingface

In [9]:
dataset_load = load_dataset("kristmh/high_vs_random")
dataset_load

DatasetDict({
    test: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'labels', '__index_level_0__'],
        num_rows: 42053
    })
    train: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'labels', '__index_level_0__'],
        num_rows: 336420
    })
    validate: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'labels', '__index_level_0__'],
        num_rows: 42052
    })
})

In [10]:
# Define a function to filter out short text entries
min_length = 1000
def filter_short_text(entry):
    return len(entry['text_clean']) > min_length
# Apply the filter function to each subset of the dataset
dataset = DatasetDict({
    'train': dataset_load['train'].filter(filter_short_text),
    'test': dataset_load['test'].filter(filter_short_text),
    'validate': dataset_load['validate'].filter(filter_short_text)
})
dataset


Filter: 100%|█████████████████████████████████| 42052/42052 [00:00<00:00, 228228.51 examples/s]


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'labels', '__index_level_0__'],
        num_rows: 62521
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'labels', '__index_level_0__'],
        num_rows: 7983
    })
    validate: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'labels', '__index_level_0__'],
        num_rows: 7938
    })
})

In [11]:
test_dataset = load_dataset(("kristmh/test_high_vs_med_low_priority"))
test_dataset

DatasetDict({
    test_separate: Dataset({
        features: ['text_clean', 'labels', 'class'],
        num_rows: 22097
    })
})

In [12]:
test_dataset = DatasetDict({
    'test_separate': test_dataset['test_separate'].filter(filter_short_text)
})
test_dataset

Filter: 100%|█████████████████████████████████| 22097/22097 [00:00<00:00, 225034.93 examples/s]


DatasetDict({
    test_separate: Dataset({
        features: ['text_clean', 'labels', 'class'],
        num_rows: 3389
    })
})

In [13]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

device(type='cuda')

## Tokenization

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

    Tokenizing the whole dataset

In [17]:
#Tokenize the dataset to the correct input for the transformer model.
def tokenize(batch):
    return tokenizer(batch["text_clean"], padding="max_length", truncation=True)

In [18]:
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset

Map: 100%|█████████████████████████████████████████| 7938/7938 [00:20<00:00, 382.22 examples/s]


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 62521
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 7983
    })
    validate: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 7938
    })
})

In [19]:
test_tokenized_dataset = test_dataset.map(tokenize, batched=True)
test_tokenized_dataset

Map: 100%|█████████████████████████████████████████| 3389/3389 [00:07<00:00, 468.71 examples/s]


DatasetDict({
    test_separate: Dataset({
        features: ['text_clean', 'labels', 'class', 'input_ids', 'attention_mask'],
        num_rows: 3389
    })
})

In [20]:
separate_test_set = test_tokenized_dataset["test_separate"]
separate_test_set = separate_test_set.remove_columns(["text_clean", "class"])
separate_test_set

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 3389
})

In [21]:
# List the columns of the dataset.
# Should be: ["attention_mask", "input_ids", "labels", "token_type_ids"]
# Remove unnecessary columns that the model does not expect.
tokenized_dataset = tokenized_dataset.remove_columns(["text_clean", "__index_level_0__", "Unnamed: 0"])

train_dataset = tokenized_dataset["train"]
print(train_dataset)
validation_dataset = tokenized_dataset["validate"]
print(validation_dataset)
test_dataset = tokenized_dataset["test"]
test_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 62521
})
Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 7938
})


Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 7983
})

## Training a classifier

In [22]:
training_args = TrainingArguments(
    output_dir=model_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    save_strategy=save_strategy,
    save_steps=save_steps,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    logging_steps=logging_steps,
)

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [24]:
trainer.train() 

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.484200,0.481475,0.759511,[0.71392177 0.79256764],[0.74788069 0.76730486],[0.68291284 0.81955056]
2,0.442500,0.486672,0.775132,[0.72685539 0.80890697],[0.7794552 0.77243917],[0.68090596 0.84898876]
3,0.303300,0.555724,0.775384,[0.74883786 0.79685542],[0.73608419 0.80818119],[0.76204128 0.7858427 ]
4,0.199800,0.738493,0.770723,[0.73321607 0.79898387],[0.75014997 0.7856212 ],[0.71702982 0.81280899]
5,0.128700,1.038155,0.771857,[0.73893614 0.79740463],[0.74311395 0.79394074],[0.73480505 0.80089888]


TrainOutput(global_step=19540, training_loss=0.32631155468424367, metrics={'train_runtime': 6093.9861, 'train_samples_per_second': 51.297, 'train_steps_per_second': 3.206, 'total_flos': 4.140997115685888e+16, 'train_loss': 0.32631155468424367, 'epoch': 5.0})

* Training loss: Difference between the predictons made by the model on the training dataset vs on the actual data.
* Validation loss: how well the model functions on unseen data.
* Accuracy: How much the model gets correct. number of correct Prediction / total number of predictions.
* F1: consider both precision and recall. 
* Precision: Accuracy of positive predictions. Percison TP = TP + FP. How often the model is correct.
* Recall: True positive rate. how many items the model gets correct from the total amount.

### Training loss decreases, validation loss increases = Overfitting

In [25]:
# Evaluate validation set
eval_result = trainer.evaluate(eval_dataset=validation_dataset)

In [26]:
for key, value in sorted(eval_result.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.7718568909045099

eval_f1 = [0.73893614 0.79740463]

eval_loss = 1.0381546020507812

eval_precision = [0.74311395 0.79394074]

eval_recall = [0.73480505 0.80089888]

eval_runtime = 49.3745

eval_samples_per_second = 160.771

eval_steps_per_second = 5.043



In [27]:
# Evaluate test data set
test_results = trainer.evaluate(eval_dataset=test_dataset)

In [28]:
for key, value in sorted(test_results.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.7509708129775774

eval_f1 = [0.71680912 0.77777778]

eval_loss = 1.1149227619171143

eval_precision = [0.72029774 0.77483296]

eval_recall = [0.71335413 0.78074506]

eval_runtime = 49.6497

eval_samples_per_second = 160.786

eval_steps_per_second = 5.035



In [29]:
trainer.save_model(model_dir + "_local") 

In [30]:
# Evaluate separate test set
separate_test_results = trainer.evaluate(eval_dataset=separate_test_set)

In [31]:
for key, value in sorted(separate_test_results.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.6444378872823842

eval_f1 = [0.72963877 0.48082723]

eval_loss = 1.6404598951339722

eval_precision = [0.60536113 0.79374111]

eval_recall = [0.91812535 0.34487021]

eval_runtime = 21.0839

eval_samples_per_second = 160.739

eval_steps_per_second = 5.028



In [32]:
from transformers import pipeline
    
classifier = pipeline("text-classification", model="./model_local")

In [33]:
classifier.model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [34]:
classifier("this does not need to be done fast")

[{'label': 'LABEL_0', 'score': 0.9929855465888977}]

In [35]:
classifier("this is super important")

[{'label': 'LABEL_1', 'score': 0.9800121188163757}]

In [36]:
classifier("this bug has super high impact on the project")

[{'label': 'LABEL_0', 'score': 0.9938083291053772}]

## Important to delete large objects to free memory 
del train_dataset

In [37]:
del validation_dataset 

In [38]:
del model

In [39]:
# Free cache
torch.cuda.empty_cache()

In [40]:
!nvidia-smi

Wed Jan 31 15:41:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  | 00000000:01:00.0 Off |                  N/A |
| 22%   32C    P2              57W / 250W |  10915MiB / 11264MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--